https://github.com/pushshift/api

In [1]:
import pandas as pd
import glob

import praw
import urllib.request
import json

from datetime import datetime
from datetime import date
import time


reddit = praw.Reddit(
    client_id="kRAJROlrIHaWMw",
    client_secret="f1EXDzFpTw1AryR_3urQ9O6I3AQ",
    user_agent="AITA_scrape",
)

In [2]:
def return_pushshift_data(subreddit, early_before, late_after, min_comments, min_score):
    link = (
        "https://api.pushshift.io/reddit/search/submission/?subreddit=%s&sort_type=created_utc&sort=desc&size=1000&before=%ss&after=%ss&num_comments=>%s&score=>%s"
        % (subreddit, early_before, late_after, min_comments, min_score)
    )
    print(link)
    with urllib.request.urlopen(link) as url:
        data = json.loads(url.read())
    data = data["data"]
    return data


def unix_to_date(unix):
    str_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(unix))
    date_time = datetime.strptime(str_time, "%Y-%m-%d %H:%M:%S")
    return date_time


def pushshift_reddit_search(subreddit, min_comments, min_score, years_ago):
    early_before = 0
    late_after = years_ago * 365 * 60 * 60 * 24
    matches = 1000
    posts = []
    while matches >= 1000:
        data = return_pushshift_data(
            subreddit, early_before, late_after, min_comments, min_score
        )
        matches = len(data)

        for match in range(0, matches):
            posts.append(
                [
                    data[match]["title"],
                    data[match]["id"],
                    unix_to_date(data[match]["created_utc"]),
                ]
            )

        if matches == 1000:
            unix_time = int(data[matches - 1]["created_utc"])
            last_time = unix_to_date(unix_time)
            now = unix_to_date(time.time())
            early_before = int((now - last_time).total_seconds() + 60)

    post_df = pd.DataFrame(
        posts, columns=["title", "id", "date"],
    )

    return post_df

In [2]:
push_df = pushshift_reddit_search(
    subreddit="AmItheAsshole", min_comments=10, min_score=25, years_ago=3
)

len(push_df)

https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&before=0s&after=94608000s&num_comments=>10&score=>25
https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&before=9932238s&after=94608000s&num_comments=>10&score=>25
https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&before=15533500s&after=94608000s&num_comments=>10&score=>25
https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&before=16307765s&after=94608000s&num_comments=>10&score=>25
https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&before=17070845s&after=94608000s&num_comments=>10&score=>25
https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&before

30104

In [3]:
push_df.to_csv("data/raw/posts_pushshift_ids.csv")

In [4]:
import collections

print(
    [
        item
        for item, count in collections.Counter(post_df["id"].tolist()).items()
        if count > 1
    ]
)

[]


In [5]:
print(push_df.shape)
push_df.head()

(30104, 3)


,title,id,date
0,AITA for renting a house and telling my neighb...,g637y9,2020-04-22 15:31:14
1,AITA my boyfriend wont let me get a breast red...,g5n4a3,2020-04-21 20:35:11
2,AITA? My parents took most of my wardrobe away...,g5hta8,2020-04-21 15:56:36
3,AITA for being my trans sisters staunchest all...,g3yzy1,2020-04-19 01:18:47
4,AITA for going on a week long vacation by myse...,g3y0xe,2020-04-19 00:11:52


In [6]:
def praw_from_subid(post_df):
    posts = []
    count = 0
    start = datetime.now()
    id_list = list(post_df["id"])

    for n in range(len(post_df)):
            post_id = id_list[n]
            post = reddit.submission(id=post_id)
            posts.append([
                post.link_flair_text,
                post.selftext,
                post.num_comments,
                post.score,
                post.upvote_ratio,
            ])
            count += 1
          
            if count % 2000 == 0:
                end = datetime.now()
                time = end - start
                print("Parsed: %s (%s)" % (count, time))
                start = datetime.now()
                praw_posts_df = pd.DataFrame(posts, columns=["verdict", "body", "num_comments", "score", "upvote_ratio"])
                praw_posts_df.to_csv("data/raw/praw_match_%s.csv" % (n+1))
                posts = []
            if n == len(post_df) - 1:
                praw_posts_df = pd.DataFrame(posts, columns=["verdict", "body", "num_comments", "score", "upvote_ratio"])
                praw_posts_df.to_csv("data/raw/praw_match_%s.csv" % (n+1))

praw_from_subid(post_df)

Parsed: 2000 (0:28:52.944591)
Parsed: 4000 (0:27:29.047117)
Parsed: 6000 (0:27:49.882244)
Parsed: 8000 (0:26:54.029158)
Parsed: 10000 (0:26:11.908075)
Parsed: 12000 (0:26:44.771090)
Parsed: 14000 (0:28:00.320595)
Parsed: 16000 (0:28:08.594987)
Parsed: 18000 (0:27:15.119710)
Parsed: 20000 (0:27:42.282699)
Parsed: 22000 (0:26:48.493822)
Parsed: 24000 (0:26:02.403901)
Parsed: 26000 (0:27:51.421883)
Parsed: 28000 (0:25:35.531361)
Parsed: 30000 (0:17:39.055438)


In [7]:
def combine_praw_sep(end_len):
    n = 0
    praw_df = pd.DataFrame()
    while n <= end_len:
        try:
            n += 2000
            posts = pd.read_csv("data/raw/praw_match_%s.csv" % n, index_col=0)
            praw_df = pd.concat([praw_df, posts])
        except:
            posts = pd.read_csv("data/raw/praw_match_%s.csv" % end_len, index_col=0)
            praw_df = pd.concat([praw_df, posts])
    praw_df.reset_index(drop=True, inplace=True)
    praw_df.to_csv("data/raw/praw_match_full.csv")
    return praw_df

praw_df = combine_praw_sep(30104)
praw_df

,verdict,body,num_comments,score,upvote_ratio
0,Not the A-hole,"I have a house I rent, and some of my neighbor...",324,4097,0.97
1,Not the A-hole,tl;dr: i have large breasts that have caused m...,1089,19436,0.90
2,Not the A-hole,"I got in trouble at school this fall, I'm a ju...",2876,26316,0.93
3,NaN,[removed],200,0,0.48
4,Asshole,I'll try to make this short. I'm (38m) and my ...,805,3845,0.91
...,...,...,...,...,...
30099,asshole,[deleted],63,39,0.96
30100,everyone sucks,So i have always gotten along perfectly with m...,88,33,0.79
30101,not the a-hole,[deleted],27,38,0.91
30102,not the a-hole,"So long story, my father passed the end of las...",37,37,0.93


In [35]:
push_df = pd.read_csv("data/raw/praw_match_full.csv", index_col=0)
praw_df = pd.read_csv("data/raw/posts_pushshift_ids.csv", index_col=0)

def combine_praw_pushshift(praw_df, push_df):
    full_df = pd.concat([push_df, praw_df], axis=1)
    full_df.drop_duplicates(subset="id", inplace=True)
    full_df.reset_index(drop=True, inplace=True)
    full_df.to_csv("data/raw/to_combine/praw_bulk_20200427.csv")
    return full_df

full_df = combine_praw_pushshift(praw_df, push_df)
print(full_df.shape)
full_df

(30104, 8)


,verdict,body,num_comments,score,upvote_ratio,title,id,date
0,Not the A-hole,"I have a house I rent, and some of my neighbor...",324,4097,0.97,AITA for renting a house and telling my neighb...,g637y9,2020-04-22 15:31:14
1,Not the A-hole,tl;dr: i have large breasts that have caused m...,1089,19436,0.90,AITA my boyfriend wont let me get a breast red...,g5n4a3,2020-04-21 20:35:11
2,Not the A-hole,"I got in trouble at school this fall, I'm a ju...",2876,26316,0.93,AITA? My parents took most of my wardrobe away...,g5hta8,2020-04-21 15:56:36
3,NaN,[removed],200,0,0.48,AITA for being my trans sisters staunchest all...,g3yzy1,2020-04-19 01:18:47
4,Asshole,I'll try to make this short. I'm (38m) and my ...,805,3845,0.91,AITA for going on a week long vacation by myse...,g3y0xe,2020-04-19 00:11:52
...,...,...,...,...,...,...,...,...
30099,asshole,[deleted],63,39,0.96,AITA for not having change when selling someth...,68pg7k,2017-05-01 23:08:08
30100,everyone sucks,So i have always gotten along perfectly with m...,88,33,0.79,AITA - Told my good friend i was pretty much s...,68nddo,2017-05-01 17:29:24
30101,not the a-hole,[deleted],27,38,0.91,AITA for defending my sister to my obsessive p...,68fji5,2017-04-30 13:51:38
30102,not the a-hole,"So long story, my father passed the end of las...",37,37,0.93,AITA for NOT sharing my inheritance? long,686xx4,2017-04-29 02:02:01


In [3]:
def unix_to_date(unix):
    str_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(unix))
    date_time = datetime.strptime(str_time, "%Y-%m-%d %H:%M:%S")
    return date_time

def praw_recent_posts():
    posts = []
    for x in ["week", "month", "year"]:
        count = 0
        for submission in reddit.subreddit('amitheasshole').top(x, limit=1000):
            posts.append([
                submission.title,
                submission.id,
                unix_to_date(submission.created_utc),
                submission.link_flair_text,
                submission.selftext,
                submission.num_comments,
                submission.score,
                submission.upvote_ratio,
            ])
            count += 1
            if count % 300 == 0:
                print(x,": Completed",count)
    post_df = pd.DataFrame(posts, columns=["title", "id", "date", "verdict", "body", "num_comments", "score", "upvote_ratio"])
    post_df.drop_duplicates(subset="id", inplace=True)
    post_df.to_csv("data/raw/to_combine/praw_recent_%s.csv" % date.today().strftime("%Y%m%d"))
    return post_df

praw_rec_df = praw_recent_posts()
len(praw_rec_df)

week : Completed 300
week : Completed 600
week : Completed 900
month : Completed 300
month : Completed 600
month : Completed 900
year : Completed 300
year : Completed 600
year : Completed 900


2691

In [4]:
def combine_new_praw_data():
    files_to_combine = glob.glob("data/raw/to_combine/*.csv")
    temp_list = []

    for filename in files_to_combine:
        df = pd.read_csv(filename, index_col=0, header=0)
        temp_list.append(df)
    full_df = pd.concat(temp_list, axis=0, ignore_index=True)

    print(len(full_df))
    full_df.drop_duplicates(subset="id", inplace=True)
    full_df.reset_index(drop=True, inplace=True)
    print(len(full_df))

    full_df.to_csv("data/interim/aita_scrapped_full.csv")
    return full_df

full_df = combine_new_praw_data()
print(full_df.shape)
full_df

37354
34961
(34961, 8)


,verdict,body,num_comments,score,upvote_ratio,title,id,date
0,Not the A-hole,"I have a house I rent, and some of my neighbor...",324,4097,0.97,AITA for renting a house and telling my neighb...,g637y9,2020-04-22 15:31:14
1,Not the A-hole,tl;dr: i have large breasts that have caused m...,1089,19436,0.90,AITA my boyfriend wont let me get a breast red...,g5n4a3,2020-04-21 20:35:11
2,Not the A-hole,"I got in trouble at school this fall, I'm a ju...",2876,26316,0.93,AITA? My parents took most of my wardrobe away...,g5hta8,2020-04-21 15:56:36
3,NaN,[removed],200,0,0.48,AITA for being my trans sisters staunchest all...,g3yzy1,2020-04-19 01:18:47
4,Asshole,I'll try to make this short. I'm (38m) and my ...,805,3845,0.91,AITA for going on a week long vacation by myse...,g3y0xe,2020-04-19 00:11:52
...,...,...,...,...,...,...,...,...
34956,Not the A-hole,When my girls were 5-6 years old they were cut...,538,12076,0.96,AITA for expecting my young daughters to be pa...,hcdcfq,2020-06-20 01:34:23
34957,Not the A-hole,"I've lost my son 2 months ago, He was only 25,...",715,11992,0.95,Aita For kicking my brother in law and his wif...,he6964,2020-06-23 02:32:11
34958,Not the A-hole,My MIL was over at our house and I’ve been fee...,583,11949,0.96,AITA for telling my MIL she needed to leave af...,hd6yby,2020-06-21 13:29:47
34959,Not the A-hole,"Hi, first time posting on here and I do feel l...",972,11823,0.97,AITA if I don’t give my sisters back even thou...,hjigc0,2020-07-01 20:33:19
